<a href="https://colab.research.google.com/github/jinijavac/NLP-Sentiment-Analysis-Project/blob/main/run_KoBERT_classifier_final_epoch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys

print(os.getcwd())

/content


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers # >= 4.x.x
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661676 sha256=ca590d5abb92b24fc766b2ab169211319e498228c5aeeb4e84773601b1625d76
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.7 MB/s eta 0:00:00


In [ ]:
### kobert 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-hlb63973/kobert-tokenizer_65a2ae751dbb41c397c72d4bddb2e508
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-hlb63973/kobert-tokenizer_65a2ae751dbb41c397c72d4bddb2e508
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=49caac2be1df5802da7a88eb5067da3467206ad0c2ae937ce7086d859081d9ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-n_qdrni6/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
model = AutoModel.from_pretrained("skt/kobert-base-v1")

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

# Data Preparation and Training

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0")

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

chatbot_data = pd.read_excel("chat_data.xlsx")

Saving chat_data.xlsx to chat_data.xlsx


In [ ]:
chatbot_data.sample(n=10)

,Sentence,Emotion
10733,"호, 7월이 코앞이군아~",놀람
5580,그리고 여자분들은 어떤 기준으로 번호 알려줄건가요 ?,놀람
20887,돌아올 대답이 너무 아플까바서기다려보려고 아직은 잊는게 너무 힘들어서,슬픔
15141,헬조선이사람 빙구만드네,분노
4643,어떻게 해야할까요 ㅠㅠㅠㅠ??,공포
665,늦잠자서 회사에 늦었다고 데려가 달라나;;;;,공포
37227,기자님 느그집 통신사 하고 뭔 관계 있는교? 고마좀 빨아라,혐오
12751,쪽팔린줄 알아라.. 속초시 관계자들은!!,분노
8175,수백개 사유가 있더라도 단 하나만 탄핵 사유에 해당하면 되는것 아닌가?........,놀람
24216,그럼 전과자 되는거에요?,중립


In [ ]:
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['언니 동생으로 부르는게 맞는 일인가요..??', '0']
['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?', '1']
['당연히 그렇게 해야지 우리나라도 판매를 중단하라', '2']
['그거들은 뒤부터 미치겠어요...', '3']
['최악의 상황중 그나마 나은 방법이네. 기분은 잡치겠지만', '4']
['  요리하는것이 숙제하는것처럼 힘든저에게 용기나게 해주시고 할수 있을것같은 희망을 주셔서감사합니다!!', '5']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']


In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

30875
7719


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))[1]
        #bert model returns 'last_hidden_state' and 'pooler_output'

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
best_acc=0.0
best_loss=99999999
ckpt_path="/content/Folder"
ckpt_name=ckpt_path+"saved_model.pt"

In [ ]:
# KoBERT 모델 학습
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e + 1, batch_id + 1, loss.data.cpu().numpy(),
                                                                      train_acc / (batch_id + 1)))
    print("epoch {} train acc {}".format(e + 1, train_acc / (batch_id + 1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        with torch.no_grad():
            out = model(token_ids, valid_length, segment_ids)
        test_loss = loss_fn(out, label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {} test loss {}".format(e + 1, test_acc / (batch_id + 1), test_loss.data.cpu().numpy()))

    if test_acc > best_acc and test_loss.data.cpu().numpy() < best_loss:
        torch.save({'epoch': e + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': test_loss.data.cpu().numpy()},
                   ckpt_name)
        best_loss = test_loss.data.cpu().numpy()
        bset_acc = test_acc

        print('current best model saved')


<ipython-input-24-6d30e2c5dbe2>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/483 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.969186544418335 train acc 0.078125
epoch 1 batch id 201 loss 1.513489007949829 train acc 0.35968594527363185
epoch 1 batch id 401 loss 1.187882661819458 train acc 0.4224205112219451
epoch 1 train acc 0.4380882888582164


<ipython-input-24-6d30e2c5dbe2>:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/121 [00:00<?, ?it/s]

...

epoch 4 test acc 0.5574307321466413 test loss 1.2562497854232788


  0%|          | 0/483 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.752691924571991 train acc 0.765625
epoch 5 batch id 201 loss 0.4772200882434845 train acc 0.7961753731343284
epoch 5 batch id 401 loss 0.4467715919017792 train acc 0.8036938902743143
epoch 5 train acc 0.8049061613373207


  0%|          | 0/121 [00:00<?, ?it/s]

epoch 5 test acc 0.5581591703750794 test loss 1.2153828144073486


In [ ]:
import random
import pandas as pd

# Assuming you have the necessary imports for BERT
# Make sure 'model', 'tok', 'vocab', 'max_len', 'device', and 'batch_size' are defined

movies_df = pd.read_excel('/content/movie.xlsx')

def recommend_movies_by_emotion(emotion, num_recommendations=3):
    emotion_movies = movies_df[movies_df['emotion'] == emotion]

    if not emotion_movies.empty:
        selected_movies = emotion_movies.sample(min(num_recommendations, len(emotion_movies)))
        recommendations = [
            f"{title} - {summary}"
            for title, summary in zip(selected_movies['title'].values, selected_movies['summary'].values)
        ]
        return recommendations
    else:
        return ["No recommended movies for this emotion."]

def predict_and_recommend(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        probabilities = F.softmax(out, dim=1)

        emotion_labels = ["공포", "놀람", "분노", "슬픔", "중립", "행복", "혐오"]

        print("감정 확률:")
        for i, prob in enumerate(probabilities[0].detach().cpu().numpy()):
            print(f"{emotion_labels[i]}: {prob * 100:.2f}%")

        max_prob_index = np.argmax(probabilities[0].detach().cpu().numpy())
        pred_emotion = emotion_labels[max_prob_index]

        print(f">> 입력하신 내용에서 {pred_emotion}의 감정이 가장 높습니다.\n")

        # Recommend 3 movies for the predicted emotion
        recommended_movies = recommend_movies_by_emotion(pred_emotion, num_recommendations=3)
        print(f">> {pred_emotion}에 어울리는 영화 추천:")
        for movie in recommended_movies:
            print(f"   - {movie}\n")



# Inference

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict_and_recommend(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 오늘 날씨 너무 좋다!
감정 확률:
공포: 0.04%
놀람: 0.19%
분노: 0.08%
슬픔: 0.11%
중립: 0.14%
행복: 99.40%
혐오: 0.04%
>> 입력하신 내용에서 행복의 감정이 가장 높습니다.

>> 행복에 어울리는 영화 추천:
   - 건축학개론 - 건축학과인 20살 승민은 건축학개론 강의에서 만난 음대생 서연에게 반해버린다. 서연과 함께 숙제를 하며 친해지게 된 승민은 자신의 마음을 고백하지 못하고 오해로 인해 그녀와 멀어지게 된다. 15년 후, 건축가가 된 승민은 서연과 재회하게 되고, 그녀는 승민에게 자신의 집을 설계해달라고 한다. 첫 작품으로 서연의 집을 짓게 된 승민은 점차 다시 그녀가 끌리기 시작한다.

   - 극한직업  - 실적 바닥으로 해체 위기에 처한 고 반장의 팀, 어느 날 이들은 국제 범죄 조직의 마약 밀반입 정황을 포착하고 잠복 수사에 뛰어든다. 이들은 범죄 조직의 아지트를 감시하기 아지트 앞 치킨집을 인수해 위장 창업을 시작한다. 그러나 곧 치킨이 유명해지기 시작하면서 수사는 뒷전이 된다.

   - 럭키 - 전문 킬러인 형욱은 목욕탕에서 비누를 밟고 넘어져 과거의 기억을 모두 잃는다. 삶에 의욕이 없던 무명배우 재성은 죽기 전 목욕탕에 들러 형욱의 키를 갖게 되고 곧 새로운 삶을 살아보기로 결심한다. 형욱과 재성의 코믹하고 기막힌 신분 변화가 시작된다.



하고싶은 말을 입력해주세요 : 진짜 화가 나. 왜 나한테만 이런 일이 생기는 거야?
감정 확률:
공포: 1.03%
놀람: 3.23%
분노: 89.73%
슬픔: 0.33%
중립: 0.43%
행복: 0.65%
혐오: 4.60%
>> 입력하신 내용에서 분노의 감정이 가장 높습니다.

>> 분노에 어울리는 영화 추천:
   - 실미도 - 월북한 아버지를 둔 강인찬은 뒷골목을 전전하는 하류 인생을 살다가 살인미수로 수감되기에 이른다. 사형을 언도받고 집행을 기다리던 그는 어느 날, 형장이 아닌 인천 외딴 부둣가에 도착해 눈을 뜬다. 